In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os

tf.keras.backend.set_floatx('float32')

#Includes ghost points for the periodic boundary condition
Nx = 203
Ny = 203

#grid size
hx = 1/(Nx-3)
hy = 1/(Ny-3)

e = 0.02

#domain size
xmin = 0.1
xmax = 1.0
ymin = 0.0
ymax = 1.0

#weight 
omega = 0.1

In [ ]:
#This function considers all the inner points and organize the nodal values for the calculation of e
def organize_interaction(A):

    first = tf.concat([tf.reshape(A[1:, 1:-1], (-1, 1)), tf.reshape(A[0:-1, 1:-1], (-1, 1))], axis=1)
    
    second = tf.concat([tf.reshape(A[1:-1, 1:], (-1, 1)), tf.reshape(A[1:-1, 0:-1], (-1, 1))], axis=1)

    input = tf.concat([first, second], axis=0)
    
    return input

In [ ]:
#Multiplying the result of f(R) with PM matrix gives c_{i,j}
def calc_PM(Nx, Ny):

    #number of divisions along x and y axes
    m = Nx - 1
    n = Ny - 1

    num_interactions = (n-1)*m + (m-1)*n
    num_inner_nodes = (m-1)*(n-1)
    
    first_gap = n-2
    second_gap = (n-2) + (n-1)*(m-2)
    
    indices = np.zeros((4*num_inner_nodes, 2))
    values = np.zeros((4*num_inner_nodes, ))
    dense_size = np.array([num_inner_nodes, num_interactions]) 
    
    current = 0
    for i in range(num_inner_nodes):
        indices[current][:] = np.array([i, i])
        values[current] = 1.
        current = current + 1
        
        indices[current][:] = np.array([i, i+1+first_gap])
        values[current] = -1.
        current = current + 1

        indices[current][:] = np.array([i, i+1+first_gap+1+second_gap + int(i/(n-1))])
        values[current] = 1.
        current = current + 1

        indices[current][:] = np.array([i, i+1+first_gap+1+second_gap+1 + int(i/(n-1))])
        values[current] = -1.
        current = current + 1

    values = tf.convert_to_tensor(values, dtype='float32')
    
    PM = tf.sparse.SparseTensor(indices, values, dense_size)
    
    return PM

In [ ]:
def create_model(input_shape, nhu=2, npl=32):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Input(input_shape))

    for _ in range(nhu):
        model.add(tf.keras.layers.Dense(npl, 
                                        activation='relu',
                                        #kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01)))
                                        kernel_initializer='glorot_normal'))
    
    model.add(tf.keras.layers.Dense(1, activation='tanh'))

    return model

In [ ]:
#Reshapes A into (Nx-2)*(Ny-2) matrix and update the ghost points
def reshape_output(Nx, Ny, A):
    A = tf.reshape(A, (Nx-2, Ny-2))
    
    #adding boundary points
    #top and buttom in the xy plane
    top = A[:, 1:2]
    bottom = A[:, -2:-1]
    
    A = tf.concat([bottom, A, top], axis=1)
    
    left = A[-2:-1, :]
    right = A[1:2, :]
    
    A = tf.concat([left, A, right], axis=0)

    return A

In [ ]:
#function to compute the energy from integral
def compute_energy(phi):
    grad2 = tf.math.square(phi[1:-1, 2:] - phi[1:-1, 0:-2])/(4.0*hx*hx) \
            + tf.math.square(phi[2:, 1:-1] - phi[0:-2, 1:-1])/(4.0*hy*hy)
            
    integral = 0.5*grad2 + (1/e**2)*0.25*tf.math.square(tf.math.square(phi[1:-1, 1:-1]) - 1)

    #riemann sum
    energy = tf.reduce_sum(hx*hy*integral)
    
    return energy


In [ ]:
#Algorithm 2 from the paper
#M is the number of message passing steps
#fR and fO are the networks
def march_forward(M, phi_in, fR, fO, PM):

    Nx = phi_in.shape[0]
    Ny = phi_in.shape[1]
    
    H = tf.zeros(phi_in.shape, dtype='float32')
    
    for _ in range(M):
        input_phi = organize_interaction(phi_in)
        input_H = organize_interaction(H)
        input1 = tf.concat([input_phi, input_H], axis=1)

        
        output1 = fR(input1)

        c = tf.sparse.sparse_dense_matmul(PM, output1)
        
        input2 = tf.concat((tf.reshape(phi_in[1:-1, 1:-1], (-1, 1)), c), axis=1)
        
        output2 = fO(input2)
        
        H = reshape_output(Nx, Ny, output2)
    
    return H

In [ ]:
def compute_loss(M, phi_in, fR, fO, PM, initial=False):
    phi_out = march_forward(M, phi_in, fR, fO, PM)
    
    energy_in = compute_energy(phi_in)
    energy_out = compute_energy(phi_out)
    
    if initial:
        loss = tf.reduce_mean(tf.math.square(phi_in - phi_out))
    else:
        loss = tf.math.reduce_mean(tf.math.square(phi_out - phi_in)) + omega*energy_out/energy_in
    
    return energy_in, energy_out, loss

In [ ]:
def initialize_phi(Nx, Ny):

    phi = np.ones((Nx, Ny))

    for i in range(Nx):
        #i-1 because we have added ghost points to store the boundary condition
        x = 0+(i-1)*hx
        for j in range(Ny):
            y = 0+(j-1)*hy
            phi[i, j] = np.sin(4*np.pi*x)*np.cos(4*np.pi*y)
    
    phi[0, :] = phi[-3, :]
    phi[-1, :] = phi[2, :]
    phi[:, 0] = phi[:, -3]
    phi[:, -1] = phi[:, 2]
    
    return tf.convert_to_tensor(phi, dtype='float32')

In [ ]:
class FONN():
    def __init__(self, M):
        self.M = M 
        self.PM = calc_PM(Nx, Ny)
        
        self.fR = create_model(input_shape=(4, ))
        self.fO = create_model(input_shape=(2, ))
        
        self.phi_0 = initialize_phi(Nx, Ny)
        self.phi = initialize_phi(Nx, Ny)
    
    def initial_training(self, epochs):
        optimizer = tf.keras.optimizers.Adam()

        for ep in range(epochs):

            noise = tf.random.normal(self.phi_0.shape, mean=0.0, stddev=0.05, dtype='float32')
            noisy_input = self.phi_0 + noise

            with tf.GradientTape() as tape:
                energy_in, energy_out, loss = compute_loss(self.M, noisy_input, self.fR, self.fO, self.PM, initial=True)
            grads = tape.gradient(loss, tape.watched_variables())
            
            optimizer.apply_gradients(zip(grads, tape.watched_variables()))
            del tape

            print("epoch = ", ep, " loss = ", loss.numpy())

    
    #K -> fine tuning steps
    def progressive_method(self, steps, K, save_plots=False, location=""):
        optimizer = tf.keras.optimizers.Adam()
        
        #open the file
        if save_plots:
            name_E = location + "/energy.txt"
            file_E = open(name_E, "w")

        for step in range(steps):
            
            for ep in range(K):
                with tf.GradientTape() as tape:
                    energy_in, energy_out, loss = compute_loss(self.M, self.phi, self.fR, self.fO, self.PM)
                
                grads = tape.gradient(loss, tape.watched_variables())

                optimizer.apply_gradients(zip(grads, tape.watched_variables()))
                del tape

            
            self.phi = march_forward(self.M, self.phi, self.fR, self.fO, self.PM)

            print("step = ", step, " energy_in = ", energy_in.numpy(), " energy_out = ", energy_out.numpy())

            #if energy gets 0 
            if np.abs(energy_out.numpy()) < 10e-5:
                data_file_name = location + "/dat_" + str(step) + ".txt"
                np.savetxt(data_file_name, np.reshape(self.phi[1:-1, 1:-1].numpy(), -1))
                fig_name = location + "/" + str(step) + ".pdf"
                title_ = "$E(\phi)$ = " + str(compute_energy(self.phi).numpy())
                plt.figure()
                plt.contourf(tf.transpose(self.phi[1:-1, 1:-1]))
                plt.colorbar()
                plt.title(title_)
                plt.savefig(fig_name)
                plt.close()
                break
            
            #To save plots and result
            if save_plots:
                result_E = str(energy_in.numpy()) + "\t" + str(energy_out.numpy()) + "\n" 
                file_E.write(result_E)
                
                #save figure
                #if step % 50 == 0:
                if True:
                    data_file_name = location + "/dat_" + str(step) + ".txt"
                    np.savetxt(data_file_name, np.reshape(self.phi[1:-1, 1:-1].numpy(), -1))
                    fig_name = location + "/" + str(step) + ".pdf"
                    title_ = "$E(\phi)$ = " + str(compute_energy(self.phi).numpy())
                    plt.figure()
                    plt.contourf(tf.transpose(self.phi[1:-1, 1:-1]))
                    plt.colorbar()
                    plt.title(title_)
                    plt.savefig(fig_name)
                    plt.close()
        
        if save_plots == True:
            file_E.close()
            
            E = np.genfromtxt(name_E, dtype='float32')
            plt.figure()
            plt.semilogy(E[:,0], linewidth=2)
            plt.grid(visible=True, which='both')
            plt.title("$E(\phi)$")
            plt.savefig(location + "/E.pdf")
            plt.close()

In [ ]:
model = FONN(2)


location = "./Lyapunov_Results"

if not os.path.exists(location):
    os.makedirs(location)

In [ ]:
model.initial_training(1000)

In [ ]:
model.progressive_method(steps=501, K=10, save_plots=True, location=location)